<a href="https://colab.research.google.com/github/eduardoscovino/Pokemon-analysis/blob/main/Pokemon_Analysis_With_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('drive/MyDrive/Kaggle/Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


## The variables of the dataset

* `Name:` Name of the pokemon
* `Type 1:` Each pokemon has a type. Ex: Grass, water, fire...
* `Type 2:` A pokemon can also have more than one type
* `Total:` The whole power of the pokemon, which is the sum of its attibutes (the following columns)
* `HP:` Hit points (or health), which is the impact a pokemon can handle before being out of combat
* `Attack:` Power of normal attacks
* `Defense:` Power of resistence against normal attacks
* `Sp. Atk:` Power of the special attack
* `Sp. Def:` Power of resistence against special attacks
* `Speed:` The criteria to define which pokemon attacks first each round
* `Generation:` From which generation is the pokemon
* `Legendary:` A boolean variable to inform if the pokemon is legendary or not

## Data Cleaning

Actually, I rather prefer working with columns written uppercased. So, let's do this now.

In [ ]:
df.columns = df.columns.str.upper()
df.head()

,#,NAME,TYPE 1,TYPE 2,TOTAL,HP,ATTACK,DEFENSE,SP. ATK,SP. DEF,SPEED,GENERATION,LEGENDARY
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [ ]:
df.loc[df.NAME.str.contains('Mega')].head()

,#,NAME,TYPE 1,TYPE 2,TOTAL,HP,ATTACK,DEFENSE,SP. ATK,SP. DEF,SPEED,GENERATION,LEGENDARY
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
12,9,BlastoiseMega Blastoise,Water,NaN,630,79,103,120,135,115,78,1,False
19,15,BeedrillMega Beedrill,Bug,Poison,495,65,150,40,15,80,145,1,False


If we look at the pokemons whose name contains 'Mega', we see that the strings before the 'Mega' are unnecessary.

So let's cut the strings before the 'Mega' in our dataset.

In [ ]:
df.NAME = df.NAME.str.replace(".*(?=Mega)", "")
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,#,NAME,TYPE 1,TYPE 2,TOTAL,HP,ATTACK,DEFENSE,SP. ATK,SP. DEF,SPEED,GENERATION,LEGENDARY
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


## Treatment for null values

In [ ]:
print(df.isnull().sum().sort_values(ascending=False) / df.shape[0])

TYPE 2        0.4825
#             0.0000
NAME          0.0000
TYPE 1        0.0000
TOTAL         0.0000
HP            0.0000
ATTACK        0.0000
DEFENSE       0.0000
SP. ATK       0.0000
SP. DEF       0.0000
SPEED         0.0000
GENERATION    0.0000
LEGENDARY     0.0000
dtype: float64


It seems only the `TYPE 2` has null values, which makes sense. These are the pokemons that don't have a dual type. In this case, let's replace the null values with "Type 1".

In [ ]:
df['TYPE 2'].fillna('Type 1', inplace=True)
df.loc[df['TYPE 2'] == 'Type 1'].head()

,#,NAME,TYPE 1,TYPE 2,TOTAL,HP,ATTACK,DEFENSE,SP. ATK,SP. DEF,SPEED,GENERATION,LEGENDARY
4,4,Charmander,Fire,Type 1,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,Type 1,405,58,64,58,80,65,80,1,False
9,7,Squirtle,Water,Type 1,314,44,48,65,50,64,43,1,False
10,8,Wartortle,Water,Type 1,405,59,63,80,65,80,58,1,False
11,9,Blastoise,Water,Type 1,530,79,83,100,85,105,78,1,False


## First impressions of the dataset

In [ ]:
print('{} rows'.format(df.shape[0]))
print('{} pokemons'.format(df.NAME.unique().size))

800 rows
800 pokemons


As we got 1 pokemon per row, makes sense to put the column NAME as the index of the dataframe. Even more, the column `#` doens't make much sense to the analysis.

In [ ]:
df.set_index('NAME', inplace=True)
df.head()

,#,TYPE 1,TYPE 2,TOTAL,HP,ATTACK,DEFENSE,SP. ATK,SP. DEF,SPEED,GENERATION,LEGENDARY
NAME,,,,,,,,,,,,
Bulbasaur,1,Grass,Poison,318,45,49,49,65,65,45,1,False
Ivysaur,2,Grass,Poison,405,60,62,63,80,80,60,1,False
Venusaur,3,Grass,Poison,525,80,82,83,100,100,80,1,False
Mega Venusaur,3,Grass,Poison,625,80,100,123,122,120,80,1,False
Charmander,4,Fire,Type 1,309,39,52,43,60,50,65,1,False


In [ ]:
df=df.drop(['#'],axis=1) # we don't need this column anymore

In [ ]:
df_summary = df.describe() # basic statistic informations
df_summary

,TOTAL,HP,ATTACK,DEFENSE,SP. ATK,SP. DEF,SPEED,GENERATION
count,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [ ]:
df.loc[(df.LEGENDARY == True) & (df.GENERATION == 1)]

,#,NAME,TYPE 1,TYPE 2,TOTAL,HP,ATTACK,DEFENSE,SP. ATK,SP. DEF,SPEED,GENERATION,LEGENDARY
156,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,1,True
157,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
158,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,1,True
162,150,Mewtwo,Psychic,Type 1,680,106,110,90,154,90,130,1,True
163,150,Mega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True
164,150,Mega Mewtwo Y,Psychic,Type 1,780,106,150,70,194,120,140,1,True
